https://stepik.org/lesson/205548/step/13

[блокнот семинара](https://colab.research.google.com/drive/1RaZT98GE7h2jhThV9S8nqFi4ooqQUwIH#scrollTo=e9Mlk2KTwBBm)

Давайте попрактикуемся с `WineNet`. Измените архитектуру так, чтобы на вход принимались все 13 признаков и проведите следующие эксперименты:

1. Поэкспериментируйте с количеством нейронов в скрытых слоях. Попробуйте поставить очень маленькое число. Существует ли пороговое значение количества скрытых нейронов, при котором обучение становится невозможным?

2. Попробуйте передавать различные значения `test_size` в функцию train_test_split. При каком значении `test_size` сеть предсказывает хуже чем `Base Rate`*? И какой `Base Rate` у датасета вин?

3. Зависит ли время обучения на одной эпохе от размера батча? Исследуйте эту зависимость.

Поделитесь своими выводами в комментариях :)



*Base Rate - значение accuracy для случая, когда модель для всех объектов предсказывает самый частотный класс в датасете

In [ ]:
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine

random.seed(4)
np.random.seed(4)
torch.manual_seed(4)
torch.cuda.manual_seed(4)
torch.cuda.manual_seed_all(4)
torch.backends.cudnn.deterministic = True

In [ ]:
# Измените архитектуру так, чтобы на вход принимались все 13 признаков
wine = load_wine()
# features =  13 # use 13 features (не нужно)

X_train, X_test, y_train, y_test = train_test_split(
    wine.data,
    wine.target,
    test_size=0.3,
    shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
wine.data.shape

(178, 13)

In [ ]:
class WineNet(torch.nn.Module):
    def __init__(self,n_input, n_hidden_neurons):
        super(WineNet, self).__init__()
        self.fc1 = torch.nn.Linear(n_input, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ3 = torch.nn.Sigmoid()
        self.fc4 = torch.nn.Linear(n_hidden_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x

n_input =  13 # choose number of input neurons
n_hidden =  3 # choose number of hidden neurons
wine_net = WineNet(n_input, n_hidden)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(wine_net.parameters(), lr=1.0e-3)

In [ ]:
batch_size = 10 # choose different batch sizes

for epoch in range(2000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]

        preds = wine_net.forward(x_batch)

        loss_value = loss(preds, y_batch)
        loss_value.backward()

        optimizer.step()

    if epoch % 10 == 0:
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.6296)
tensor(0.6111)
tensor(0.6111)
tensor(0.5926)
tensor(0.5926)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6111)
tensor(0.6111)
tensor(0.6111)
tensor(0.6296)
tensor(0.6111)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6481)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6481)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6481)
tensor(0.6481)
tensor(0.6296)
tensor(0.6296)
tensor(0.6667)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6296)
tensor(0.6481)
tensor(0.6296)
tensor(0.6667)
tensor(0.6481)
tensor(0.6296)
tensor(0.6296)
tensor(0.6667)
tensor(0.6481)
tensor(0.6481)
tensor(0.6667)
tensor(0.6667)
tensor(0.6296)
tensor(0.6

In [ ]:
print(wine_net.fc1.in_features, np.asarray((test_preds == y_test).float().mean()) > 0.8)
np.asarray((test_preds == y_test).float().mean())
# need to get 13 True

13 True


array(0.9074074, dtype=float32)

In [ ]:
#@title Эксперимент по изменению количества нейронов в скрытых слоях
result = []
for n_hidden in range(3, 13):
    wine_net = WineNet(n_input, n_hidden)
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(wine_net.parameters(), lr=1.0e-3)
    batch_size = 10

    for epoch in range(2000):
        order = np.random.permutation(len(X_train))
        for start_index in range(0, len(X_train), batch_size):
            # обнуляем оптимайзер
            optimizer.zero_grad()

            batch_indexes = order[start_index:start_index+batch_size]

            x_batch = X_train[batch_indexes]
            y_batch = y_train[batch_indexes]

            preds = wine_net.forward(x_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()

        if epoch % 10 == 0:
            test_preds = wine_net.forward(X_test)
            test_preds = test_preds.argmax(dim=1)
            print((test_preds == y_test).float().mean())

    result.append((n_hidden, (test_preds == y_test).float().mean()))

result

tensor(0.3148)
tensor(0.3333)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3519)
tensor(0.3

[(3, tensor(0.3519)),
 (4, tensor(0.6481)),
 (5, tensor(0.6296)),
 (6, tensor(0.3519)),
 (7, tensor(0.9630)),
 (8, tensor(0.3519)),
 (9, tensor(0.6667)),
 (10, tensor(0.9630)),
 (11, tensor(0.3519)),
 (12, tensor(0.9074))]